In [76]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
from datetime import datetime

In [77]:
datetime.now()

datetime.datetime(2020, 6, 12, 13, 29, 32, 482830)

In [78]:
query = '筆電'
# 每次加載分頁有20筆資料，可以依需求增加/減少
pages = 10

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}

df = []
for page in list(range(1, pages)):
    url = 'https://ecshweb.pchome.com.tw/search/v3.3/all/results?q={}&page={}&sort=sale/dc'.format(query, page)
    # Get Prod List
    resp = requests.get(url,headers=headers)
    try:
        for num, prodid in enumerate(resp.json()['prods']):
#             print(prodid)
            try:
                # basic info
                ecpai = 'https://mall.pchome.com.tw/ecapi/ecshop/prodapi/v2/prod/{}&fields=Seq,Id,Name,Nick,Store,PreOrdDate,SpeOrdDate,Price,Discount,Pic,Weight,ISBN,Qty,Bonus,isBig,isSpec,isCombine,isDiy,isRecyclable,isCarrier,isMedical,isBigCart,isSnapUp,isDescAndIntroSync,isFoodContents,isHuge,isEnergySubsidy,isPrimeOnly,isWarranty,isLegalStore,isOnSale,isPriceTask,isBidding,isETicket&_callback=jsonp_prod&1587196620'.format(prodid['Id'])
                resp2 = requests.get(ecpai,headers=headers)
                data2 = re.sub('try{jsonp_prod\(|\}\);\}catch\(e\)\{if\(window.console\)\{console.log\(e\)\;\}','',resp2.text)
                data2 = json.loads(data2)[prodid['Id']+'-000']
                ecapi2 = 'https://ecapi.pchome.com.tw/cdn/ecshop/prodapi/v2/prod/{}/desc&fields=Id,Stmt,Equip,Remark,Liability,Kword,Slogan,Author,Transman,Pubunit,Pubdate,Approve&_callback=jsonp_desc'.format(prodid['Id'])
                resp3 = requests.get(ecapi2,headers=headers)
                data3 = re.sub('try{jsonp_desc\(|\}\);\}catch\(e\)\{if\(window.console\)\{console.log\(e\)\;\}','',resp3.text)
                data3 = json.loads(data3)[prodid['Id']]
                
                ndf = DataFrame([{'Id':data3['Id'],
                                  '名稱':data2['Name'],
                                  'Nick':data2['Nick'],
                                  'Store':data2['Store'],
                                  '原價':data2['Price']['M'],
                                  '特價':data2['Price']['P'],
                                  'Stmt':data3['Stmt'],
                                  'Remark': data3['Remark'],
                                  'Kword': data3['Kword'],
                                  '描述': data3['Slogan'],
                                  'Updatetime':datetime.now()}])
                print(ndf['名稱'])
                df.append(ndf)
                sleep(4) # 後面再改成2/3秒
            except:
                pass            
    except:
        print('No More data')
    break
        
df = pd.concat(df, ignore_index=True)
df

ValueError: No objects to concatenate

In [73]:
data3['Slogan']

'<font size=2><LI>LCD尺寸：14’//300nits//FHD 1920x1080 16:9//Touch screen//sRGB: 100%//Wide View\r\n<font color=red><LI>處理器：Intel® Core™ i7-10510U 1.8 GHz</font>\r\n<LI>記憶體：8GB LPDDR3 on board\r\n<font color=red><LI>顯卡：NVIDIA MX 250 2G獨顯\r\n<LI>硬碟："512GB M.2 NVMe™ PCIe® SSD with 32GB Intel® Optane™ Memory</font>\r\n<LI>其他：NonaEdge極窄邊框、NumberPad 2.0\r\n<LI>網路：802.11ax+Bluetooth 5.0 (Dual band) 2*2\r\n<LI>重量：1.4kg\r\n<LI>作業系統： 64 Bits Windows 10 Home\r\n<LI>保固：2年全球保固 LCD無亮點保固\r\n\r\n<a href=" //a.ecimg.tw/img/projects/personal/v0/upload_file/US00010362/Office-introduce.jpg "target=”_blank”><img src="https://a.ecimg.tw/img/projects/personal/v0/upload_file/US00010362/office/2019-1.jpg "alt="↑買NB加購Office 詳細請點"></a>\r\n\r\n<a href="#規格說明"><img src="https://a.ecimg.tw/img/projects/personal/v0/upload_file/US00000208/add/icon.jpg">本商品詳細規格</a></a>\r\n\r\n\r\n<B><font color="#0072ff" style="background-color: #ffff99">館長小叮嚀：儲值購買最划算～</font></B>\r\n<a href="https://www.pchomeec.tw/activity/AC00051773">

In [59]:
ecpai= 'https://ecapi.pchome.com.tw/cdn/ecshop/prodapi/v2/prod/DHAS1U-A900AMWRG/desc&fields=Id,Stmt,Equip,Remark,Liability,Kword,Slogan,Author,Transman,Pubunit,Pubdate,Approve&_callback=jsonp_desc'

In [55]:
resp2 = requests.get(ecpai,headers=headers)

In [56]:
data = 

In [57]:
data

'{"DHAFHQ-A900AMA95":{"Id":"DHAFHQ-A900AMA95","Stmt":"<a name=\\"\\u898f\\u683c\\u8aaa\\u660e\\"> ASUS F571GT-0331K8300H \\u661f\\u591c\\u9ed1 \\u8a73\\u7d30\\u898f\\u683c\\u8868<\\/a>\\r\\n--------------------------------------------\\r\\n\\u25cf\\u4e3b\\u8981\\u898f\\u683c: \\r\\n--------------------------------------------\\r\\nLCD\\u5c3a\\u5bf8: 15.6\'\\/\\/LED Backlit\\/\\/250nits\\/\\/FHD (1920 x 1080) 16:9\\/\\/Anti-glare display\\/\\/NTSC: 45%\\/\\/IPS-level Panel\\r\\n<font color=\\"RED\\">CPU: Intel\\u00ae Core\\u2122 i5-8300H Processor 2.3 GHz (8M Cache, up to 4.0 GHz)<\\/font>\\r\\n\\u5167\\u5efa\\u8a18\\u61b6\\u9ad4:4GB DDR4 on board\\r\\n<font color=\\"RED\\">\\u786c\\u789f: \\"1TB SATA 5400RPM 2.5\\"\\" HDD+256GB M.2 NVMe\\u2122 PCIe\\u00ae 3.0  SSD\\"\\r\\n\\u986f\\u793a\\u6676\\u7247:NVIDIA GeForce GTX1650 4G\\u7368\\u986f <\\/font>\\r\\n\\u5149\\u789f\\u6a5f:\\u7121\\r\\n\\u4fdd\\u56fa:2\\u5e74\\u5168\\u7403\\u4fdd\\u56fa LCD\\u7121\\u4eae\\u9ede\\u4fdd\\u56fa\\r\\n\\

In [51]:
json.loads(data)['DHAFGB-A900A6R5O']

{'Id': 'DHAFGB-A900A6R5O',
 'Stmt': '<a name="規格說明"> ASUS VivoBook S15 S531FL-0122B8265U 藍不倒 詳細規格表</a>\r\n--------------------------------------------\r\n●主要規格: \r\n--------------------------------------------\r\nLCD尺寸: 15.6\'//Narrow border//250nits//FHD /Anti-Glare//NTSC: 45%//Wide View\r\n<font color="RED">CPU: Intel® Core™ i5-8265U Processor 1.6 GHz (6M Cache, up to 3.4 GHz)\r\n顯示晶片:Nvidia MX 250 2G獨顯</font>\r\n內建記憶體: DRAM DDR4 8G (Max 16G)\r\n<font color="RED">HDD硬碟:512G PCIE SSD</font>\r\n光碟機:無\r\n保固:2年全球保固 LCD無亮點保固\r\n作業系統:64 Bits Windows 10 Home \r\n--------------------------------------------\r\n●其他規格: \r\n--------------------------------------------\r\n尺寸：35.7(W) x 23.0(D) x 1.85 ~ 1.85 (H) cm\r\n重量：1.8 KG \r\nWLAN無線網路:802.11 AC\r\nWEBCAM網路攝影機:HD網路攝影機\r\n藍芽:Bluetooth 4.2 (Dual band) 2*2\r\n--------------------------------------------\r\n●輸入輸出介面(I/O):\r\n--------------------------------------------\r\nUSB 3.0 x 1\r\nUSB 2.0 x 2\r\nUSB 3.1 Type C x 1(Gen 1)\r\n內建多合一讀卡機\r\nHDMI\